# Import packages

In [1]:
#!pip install opencv-python
#!pip install pytesseract
!pip install imutils
#!pip install tensorflow
#!aws sts get-caller-identity - not a package, but tells you role in AWS
!pip install torch
!pip install fastai
!pip install -Uqq fastbook
!pip install graphviz

You should consider upgrading via the 'c:\users\sam bradley\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sam bradley\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sam bradley\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
voila 0.2.7 requires nbconvert<7,>=6.0.0, but you have nbconvert 5.6.1 which is incompatible.


You should consider upgrading via the 'c:\users\sam bradley\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import PIL #
import numpy as np
import glob
# nltk.download('punkt')
#import nltk
import re
import cv2
import imutils
import tensorflow as tf
import os
#from nltk.tokenize import word_tokenize, sent_tokenize
import random
import pytesseract
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC, SVC
import pickle
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, f1_score, make_scorer, accuracy_score, recall_score, precision_score, roc_auc_score
import pandas as pd
import torch
import matplotlib.pyplot as plt
import fastai
import fastbook
from fastbook import *
from fastai.vision import *
from fastai.vision.widgets import *
fastbook.setup_book()

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


### Bag of words functions

In [3]:
# augments the image to make it easier to extract text
def process_img(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # convert to grayscale
    img = imutils.rotate_bound(img, 90) # rotate 
    img = cv2.resize(img, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC) # make 3 times larger
    img = cv2.GaussianBlur(img,(3,3),0) # add gaussian blur after enlarging
    return img
    
# extracts the text and returns a set of unique words extracted from the text
def extract_words(img):
    raw_words = pytesseract.image_to_string(img)
    # Removes non-alphanumeric characters and underscores and makes lowercase
    pattern = re.compile('[\W_]+')
    words = pattern.sub(' ', raw_words).lower()

    tokens = set(word_tokenize(words))
    #print('extracted words:', words)
    return tokens

def clean_words(word_data):
    # remove words that are 2 chars or lower to reduce noise
    word_data_remove_2char = [ x for x in word_data if len(x) > 2]
    #word_data_remove_2char_google = [ set(x for x in r if len(x) > 2) for r in word_data_google]

    # format list of token sets into a list of strings which are tokens joined by spaces
    # [ set('a', 'b'), set ( 'c', 'd', 'e')]  into   ['a b', 'c d e']
    word_data_formatted = ' '.join(word_data_remove_2char)
    #word_data_google_formatted = [ ' '.join(sets) for sets in word_data_remove_2char_google]
    return word_data_formatted

def file_to_sentence(a):
    # given a file path, apply all transforms and returns a single string
    # with all extracted words over 2 characters
    a = load_img(a)
    a = process_img(a)
    a = extract_words(a)
    a = clean_words(list(a))
    return a    

# Creating an app

## Functions

In [4]:
def img_to_sentence(pil_img):
    # given a file path, apply all transforms and returns a single string
    # with all extracted words over 2 characters
    
    a = np.array(pil_img)
    a = process_img(a)
    a = extract_words(a)
    a = clean_words(list(a))
    return a    

def bow_predictor_app_inp(pil_img):
    # input either a single file string or a list of file strings to be processed
    # returns a prediction (1=KM, 0=0ther), the probability the image is KM, and the max probability ( probability of predicted class -
    # so if prediction is 0=other, max prob will tell you the probabilty that the curve is 0)
    
    # Load linear support vector classifier 
    with open("Y. model - bag of words2.pkl", 'rb') as file:  
        bow_model = pickle.load(file)

    # Load count vectorizer
    with open('Y. Count vectorizer vocab.pkl', 'rb') as f:
        vectorizer_vocab = pickle.load(f)
    
    # get an error if only one img is passed and use map functionality
#     try:
#         X_bow = list(map(file_to_sentence, X_input))
#     except:
#         # if single image, make it look like a list
#         X_bow = list(map(file_to_sentence, [X_input]))

    X_bow = [img_to_sentence(pil_img)]
        
    vectorizer = CountVectorizer(vocabulary=vectorizer_vocab)
    X_bow = vectorizer.transform(X_bow)
    
    bow_pred = bow_model.predict(X_bow)
    bow_prob = bow_model.predict_proba(X_bow)
    bow_max_prob = [max(x) for x in bow_prob]
    bow_prob = bow_prob[:,1] # probability the graph is KM
    
    return bow_pred, bow_prob, bow_max_prob

def fastai_predictor_app_inp(pil_img):
    # input either a single file string or a list of file strings to be processed
    # returns a prediction (1=KM, 0=0ther), the probability the image is KM, and the max probability ( probability of predicted class -
    # so if prediction is 0=other, max prob will tell you the probabilty that the curve is 0)
    X_fastai = pil_img

    pred_tensor,pred_bool, fastai_pred = learn_inf.predict(X_fastai)

    fastai_pred = pred_tensor[0].item() # 1=KM present 0=KM not present
    fastai_prob = prob_tensor[0].item() # prob graph is KM
    # fastai_max_prob = max(prob_tensor).numpy() # returns max values from (0 prob, 1prob) tensor for each input

    return fastai_pred, fastai_prob


def ensemble_predictor_app_inp(pil_img):
    # Function that calculates the ensemble prediction by averaging probablities
    # from BOW and fastai model. returns prob and prediction
    
    # Calculate probabilities from 2 models
    fastai_pred, fastai_prob, fastai_max_prob = fastai_predictor_app_inp(pil_img)
    bow_pred, bow_prob, bow_max_prob= bow_predictor_app_inp(pil_img)
    
    a = bow_prob
    b= fastai_prob
    c= [a,b]
    
    ensemble_prob = np.mean(c, axis=0)
    ensemble_pred = np.round(ensemble_prob)
    return ensemble_pred,ensemble_prob 

## Code for app

There must be something wrong with the fastai_img_inp code. It works with all predictions that dont use that func. have a look

In [5]:
learn_inf = load_learner('KM_classifier_multilabel_7_epoch.pkl')

btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
        
    # Choose classification method
    
    # Bow prediction -------------------------------------------------
    #pred, prob, max_prob = bow_predictor_app_inp(img)
    
    # Fastai prediction ----------------------------------------------
    pred, prob, max_prob = fastai_predictor_app_inp(img) # not working
    
    # Ensemble prediction --------------------------------------------
    #pred, prob = ensemble_predictor_app_inp(img) 
    
    lbl_pred.value = f'Prediction: {pred}, Prob: {prob}'

btn_run.on_click(on_click_classify)

AttributeError: Can't get attribute 'get_x' on <module '__main__'>

In [5]:
VBox([widgets.Label('Please upload an image'), 
      btn_upload, btn_run, out_pl, lbl_pred])

# Turn it into a real app

In [34]:
!pip install pyqt5 
!pip install pyqtwebengine 
!pip install nbconvert 
!pip install voila

In [35]:
import voila

In [36]:
!jupyter serverextension enable voila --sys-prefix

Enabling: voila
- Writing config: c:\users\sam bradley\appdata\local\programs\python\python39\etc\jupyter
    - Validating...
      voila 0.2.7 ok


In [21]:
!jupyter

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]
jupyter: error: one of the arguments --version subcommand --config-dir --data-dir --runtime-dir --paths is required
